In [1]:
import os

if 'notebook' in os.getcwd():
    os.chdir('..')
os.listdir('data')

['ind49_m_vw_rets.csv',
 'F-F_Research_Data_Factors_daily.CSV',
 'sample_prices.csv',
 'ind30_m_vw_rets.csv',
 'Portfolios_Formed_on_ME_monthly_EW.csv',
 'ind30_m_size.csv',
 'ind30_m_nfirms.csv',
 'ind49_m_ew_rets.csv',
 'F-F_Research_Data_Factors_m.csv',
 'F-F_Research_Data_Factors.CSV',
 'ind30_m_ew_rets.csv',
 'brka_d_ret.csv',
 'ind49_m_nfirms.csv',
 'ind49_m_size.csv',
 'BRK-A.csv',
 'edhec-hedgefundindices.csv']

### 1. Read data
Use the EDHEC Hedge Fund Indices data set that we used in the lab assignment as well as in the previous week’s assignments. Load them into Python and perform the following analysis based on data since 2000 (including all of 2000)

In [93]:
import pandas as pd

hfi = pd.read_csv("data/edhec-hedgefundindices.csv", index_col=0)
hfi.index = pd.DatetimeIndex(hfi.index, dayfirst=True)

hfi = hfi/100
hfi = hfi.loc['2000':]
hfi.head()

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
2000-01-31,0.0227,0.0128,0.0088,0.0077,0.0075,0.0088,0.0041,0.0021,0.0075,0.0143,0.0173,0.0427,0.0169
2000-02-29,0.0267,-0.0022,0.0421,0.0528,0.0253,0.0346,0.0097,0.0408,0.0699,0.0239,0.0185,-0.1340,0.0666
2000-03-31,0.0243,-0.0138,0.0103,0.0318,0.0134,0.0069,-0.0061,-0.0104,0.0006,0.0131,0.0163,-0.0230,0.0039
2000-04-30,0.0223,-0.0241,-0.0101,-0.0541,0.0168,-0.0059,-0.0006,-0.0304,-0.0201,0.0188,0.0092,0.1028,-0.0269
2000-05-31,0.0149,0.0114,-0.0132,-0.0433,0.0062,-0.0034,0.0107,-0.0070,-0.0097,0.0146,0.0080,0.0704,-0.0122


## Gaussian VaR with Cornish-Fisher Adjustment
1. What was the Monthly Parametric Gaussian VaR at the 1% level (as a +ve number) of the Distressed Securities strategy?
2. Use the same data set at the previous question. What was the 1% VaR for the same strategy after applying the Cornish-Fisher Adjustment?
3. Use the same dataset as the previous question. What was the Monthly Historic VaR at the 1% level (as a +ve number) of the Distressed Securities strategy?

In [104]:
import numpy as np
from scipy.stats import norm, kurtosis

def skewness(r):
    """
    Alternative to scipy.stats.skew()
    Computes the skewness of the supplied Series or DataFrame
    Returns a float or a Series
    """
    demeaned_r = r - r.mean()
    # use the population standard deviation, so set dof=0
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**3).mean()
    return exp/sigma_r**3


def kurtosis(r):
    """
    Alternative to scipy.stats.kurtosis()
    Computes the kurtosis of the supplied Series or DataFrame
    Returns a float or a Series
    """
    demeaned_r = r - r.mean()
    # use the population standard deviation, so set dof=0
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**4).mean()
    return exp/sigma_r**4


def var_gaussian(r, level=5, modified=False):
    """
    Returns the Parametric Gauusian VaR of a Series or DataFrame
    If "modified" is True, then the modified VaR is returned,
    using the Cornish-Fisher modification
    """
    # compute the Z score assuming it was Gaussian
    z = norm.ppf(level/100)
    if modified:
        # modify the Z score based on observed skewness and kurtosis
        s = skewness(r)
        k = kurtosis(r)
        z = (z +
                (z**2 - 1)*s/6 +
                (z**3 -3*z)*(k-3)/24 -
                (2*z**3 - 5*z)*(s**2)/36
            )
    return -(r.mean() + z*r.std(ddof=0))

def var_historic(r, level=5):
    """
    Returns the historic Value at Risk at a specified level
    i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100-level) percent are above
    """
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        return -np.percentile(r, level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")  

In [105]:
distressed_securities_var = var_gaussian(
    hfi.loc[:,'Distressed Securities'],
    level=1,
    modified=False
)
(distressed_securities_var *100).round(2)

3.14

In [106]:
distressed_securities_var = var_gaussian(
    hfi.loc[:,'Distressed Securities'],
    level=1,
    modified=True
)
(distressed_securities_var *100).round(2)

4.97

In [107]:
distressed_securities_var = var_historic(
    hfi.loc[:,'Distressed Securities'],
    level=1
)
(distressed_securities_var *100).round(2)

4.26

## MSR, EW, GMV

Next, load the 30 industry return data using the erk.get_ind_returns() function that we developed during the lab sessions. For purposes of the remaining questions, use data during the 5 year period 2013-2017 (both inclusive) to estimate the expected returns as well as the covariance matrix. To be able to respond to the questions, you will need to build the MSR, EW and GMV portfolios consisting of the “Books”, “Steel”, "Oil", and "Mines" industries. Assume the risk free rate over the 5 year period is 10%.


In [49]:
industries = ["Books", "Steel", "Oil", "Mines"]
rf = 5

def get_ind_returns():
    """
    Load and format the Ken French 30 Industry Portfolios Value Weighted Monthly Returns
    """
    ind = pd.read_csv("data/ind30_m_vw_rets.csv", header=0, index_col=0)/100
    ind.index = pd.to_datetime(ind.index, format="%Y%m").to_period('M')
    ind.columns = ind.columns.str.strip()
    return ind

ind = get_ind_returns()
ind = ind.loc[:,industries]
display(ind.head(2))

,Books,Steel,Oil,Mines
1926-07,0.1097,0.0407,-0.0140,0.0564
1926-08,0.1001,0.0217,0.0369,0.0055


In [50]:
def annualize_rets(r, periods_per_year):
    """
    Annualizes a set of returns
    We should infer the periods per year
    but that is currently left as an exercise
    to the reader :-)
    """
    compounded_growth = (1+r).prod()
    n_periods = r.shape[0]
    return compounded_growth**(periods_per_year/n_periods)-1

def annualize_vol(r, periods_per_year):
    """
    Annualizes the vol of a set of returns
    We should infer the periods per year
    but that is currently left as an exercise
    to the reader :-)
    """
    return r.std()*(periods_per_year**0.5)

def sharpe_ratio(r, riskfree_rate, periods_per_year):
    """
    Computes the annualized sharpe ratio of a set of returns
    """
    # convert the annual riskfree rate to per period
    rf_per_period = (1+riskfree_rate)**(1/periods_per_year)-1
    excess_ret = r - rf_per_period
    ann_ex_ret = annualize_rets(excess_ret, periods_per_year)
    ann_vol = annualize_vol(r, periods_per_year)
    return ann_ex_ret/ann_vol


er = annualize_rets(ind["2013":"2017"], 12)
display(er)

cov = ind["2013":"2017"].cov()
display(cov)

Books    0.095312
Steel    0.100716
Oil      0.025275
Mines    0.005136
dtype: float64

,Books,Steel,Oil,Mines
Books,0.002720,0.001940,0.001637,0.001539
Steel,0.001940,0.004391,0.002122,0.003391
Oil,0.001637,0.002122,0.002697,0.002179
Mines,0.001539,0.003391,0.002179,0.005232


1. What is the weight of Steel in the EW Portfolio? `25%`
2. What is the weight of the largest component of the MSR portfolio?   
3. Which of the 4 components has the largest weight in the MSR portfolio?
4. How many of the components of the MSR portfolio have non-zero weights?  
5. What is the weight of the largest component of the GMV portfolio?
6. Which of the 4 components has the largest weight in the GMV portfolio?    
7.  How many of the components of the GMV portfolio have non-zero weights?    

In [58]:
from scipy.optimize import minimize

def portfolio_return(weights, returns):
    """
    Computes the return on a portfolio from constituent returns and weights
    weights are a numpy array or Nx1 matrix and returns are a numpy array or Nx1 matrix
    """
    return weights.T @ returns

def portfolio_vol(weights, covmat):
    """
    Computes the vol of a portfolio from a covariance matrix and constituent weights
    weights are a numpy array or N x 1 maxtrix and covmat is an N x N matrix
    """
    return (weights.T @ covmat @ weights)**0.5

def msr(riskfree_rate, er, cov):
    """
    Returns the weights of the portfolio that gives you the maximum sharpe ratio
    given the riskfree rate and expected returns and a covariance matrix
    """
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),) * n # an N-tuple of 2-tuples!
    # construct the constraints
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1
    }
    def neg_sharpe(weights, riskfree_rate, er, cov):
        """
        Returns the negative of the sharpe ratio
        of the given portfolio
        """
        r = portfolio_return(weights, er)
        vol = portfolio_vol(weights, cov)
        return -(r - riskfree_rate)/vol
    
    weights = minimize(neg_sharpe, init_guess,
                       args=(riskfree_rate, er, cov), method='SLSQP',
                       options={'disp': False},
                       constraints=(weights_sum_to_1,),
                       bounds=bounds)
    return weights.x

def gmv(cov):
    """
    Returns the weights of the Global Minimum Volatility portfolio
    given a covariance matrix
    """
    n = cov.shape[0]
    return msr(0, np.repeat(1, n), cov)

All industries would have weight of `25.0`% in EW portfolio.  
Largest MSR component is `Steel` with weight of `100.0`%.  
Only `1` component has non zero weights.

In [64]:
msr_weights = pd.Series(msr(0.10, er, cov).round(2), index=industries)
msr_weights*100

Books      0.0
Steel    100.0
Oil        0.0
Mines      0.0
dtype: float64

Largest weight in GMV portfolio is `47.70` .  
Largest component is `Books` .  
Only `3` components have non-zero weights.  

In [63]:
gmv_weights = pd.Series(gmv(cov).round(4), index=industries)
gmv_weights*100

Books    47.70
Steel     0.00
Oil      43.41
Mines     8.89
dtype: float64

In [66]:
er = annualize_rets(ind["2018"], 12)
display(er)

cov = ind["2018"].cov()
display(cov)

/home/jeremy/anaconda3/envs/regime/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


Books   -0.252523
Steel   -0.273127
Oil     -0.181373
Mines   -0.307972
dtype: float64

/home/jeremy/anaconda3/envs/regime/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  after removing the cwd from sys.path.


,Books,Steel,Oil,Mines
Books,0.002799,0.002098,0.002558,0.001629
Steel,0.002098,0.004027,0.003473,0.001843
Oil,0.002558,0.003473,0.005296,0.001961
Mines,0.001629,0.001843,0.001961,0.002059


In [68]:
msr_weights.values

array([0., 1., 0., 0.])

## OOS results of GMV and MSR

Assume two different investors invested in the GMV and MSR portfolios at the start of 2018 using the weights we just computed. Compute the annualized volatility of these two portfolios over the next 12 months of 2018?

(Hint: Use the portfolio_vol code we developed in the lab and use ind[“2018”][l].cov() to compute the covariance matrix for 2018, assuming that the variable ind holds the industry returns and the variable l holds the list of industry portfolios you are willing to hold. Don’t forget to annualize the volatility)

In [75]:
(portfolio_vol(msr_weights.values, cov)*np.sqrt(12) *100).round(2)

21.98

What would be the annualized volatility over 2018 using the weights of the GMV portfolio? 

(Reminder and Hint: Use the portfolio_vol code we developed in the lab and use ind[“2018”][l].cov() to compute the covariance matrix for 2018, assuming that the variable ind holds the industry returns and the variable l holds the list of industry portfolios you are willing to hold. Don’t forget to annualize the volatility) 

In [74]:
(portfolio_vol(gmv_weights.values, cov)*np.sqrt(12) *100).round(2)

18.97